# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация

Дата выдачи: 30.01.2025

Мягкий дедлайн: 23:59MSK 16.02.2025

Жесткий дедлайн: 23:59MSK 23.02.2025

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Использование генеративных языковых моделей разрешено только в случае явного указания на это. Необходимо прописать (в соответствующих пунктах, где использовались, либо в начале/конце работы):
- какая языковая модель использовалась
- какие использовались промпты и в каких частях работы
- с какими сложностями вы столкнулись при использовании генеративных моделей, с чем они помогли больше всего

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-08-random-features-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Fourier Features (RFF, также в литературе встречается название Random Kitchen Sinks) для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import time

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [ ]:
import numpy as np

# 1 Способ
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()

# 2 Способ (если первый не работает)
# from sklearn.datasets import fetch_openml
# def load_fashion_mnist():
#     X, y = fetch_openml('Fashion-MNIST', version=1, return_X_y=True, as_frame=False)
#     X = X.reshape(-1, 28, 28).astype('uint8')
#     y = y.astype('int64')
#     x_train, x_test = X[:60000], X[60000:]
#     y_train, y_test = y[:60000], y[60000:]
#     return (x_train, y_train), (x_test, y_test)
# (x_train_pics, y_train), (x_test_pics, y_test) = load_fashion_mnist()




x_train = x_train_pics.reshape(y_train.shape[0], -1)
x_test = x_test_pics.reshape(y_test.shape[0], -1)

__Задание 0. (0.25 баллов)__

**Вопрос:** зачем в алгоритме нужен метод главных компонент?

**Ответ:** Если данные содержат много коррелированных или нерелевантных признаков, PCA помогает избавиться от них, тем самым помогает избавиться от переобучения.

__Задание 1. (3 балла)__

Реализуйте алгоритм, описанный выше. Можете воспользоваться шаблоном класса в `homework_practice_08_rff.py` (допишите его и исправьте несостыковки в классе пайплайна) или написать свой интерфейс.

Ваша реализация должна поддерживать следующие опции:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [ ]:
from homework_practice_08_rff_SargsyanNarek import RFFPipeline, RandomFeatureCreator

In [ ]:
from sklearn.metrics import accuracy_score

pipeline = RFFPipeline(n_features=1000, new_dim=50, use_PCA=True, feature_creator_class=RandomFeatureCreator)

start = time.time() #Начало времени обучения

pipeline.fit(x_train, y_train)

finish = time.time() # Окончение обучения

startp = time.time() #Начало времени предсказания

y_pred = pipeline.predict(x_test)

finishp = time.time() # Окончение предсказания

print(f"Test score = {accuracy_score(y_test, y_pred)}, fit time {finish-start:.3} second, predict time {finishp-startp:.2} second")

Test score = 0.8593, fit time 55.0 second, predict time 0.73 second


__Задание 2. (2.5 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучите градиентный бустинг. Используйте одну из реализаций CatBoost/LightGBM/XGBoost.

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

Начнем сперва с SVM

In [ ]:
from sklearn.svm import SVC

Сперва попробуем линейный SVM

In [ ]:
# Возьмем часть выборки
np.random.seed(152)
indices_train = np.random.choice(x_train.shape[0], size=(x_train.shape[0] // 8), replace=False)
indices_test = np.random.choice(x_test.shape[0], size=(x_test.shape[0] // 8), replace=False)

x_train_sample = x_train[indices_train]
x_test_sample = x_test[indices_test]
y_train_sample = y_train[indices_train]
y_test_sample = y_test[indices_test]

In [ ]:
svm_lin = SVC(kernel='linear')
start = time.time() #Начало времени обучения

svm_lin.fit(x_train_sample, y_train_sample)

finish = time.time() # Окончение обучения

startp = time.time() #Начало времени предсказания

y_pred = svm_lin.predict(x_test_sample)

finishp = time.time() # Окончение предсказания

print(f'Test score for linear SVM = {accuracy_score(y_test_sample, y_pred)}, fit time {finish-start:.2} second, predict time {finishp-startp:.2} second')

Test score for linear SVM = 0.784, fit time 7.7 second, predict time 1.9 second


Теперь ядровой SVM

In [ ]:
svm_rbf = SVC(kernel='rbf')

start = time.time() #Начало времени обучения

svm_rbf.fit(x_train_sample, y_train_sample)

finish = time.time() # Окончение обучения

startp = time.time() #Начало времени предсказания

y_pred = svm_rbf.predict(x_test_sample)

finishp = time.time() # Окончение предсказания

print(f'Test score for RBF SVM = {accuracy_score(y_test_sample, y_pred)}, fit time {finish-start:.2} second, predict time {finishp-startp:.2} second')

Test score for RBF SVM = 0.8312, fit time 6.0 second, predict time 3.6 second


Теперь используем GB

In [ ]:
!pip3 install catboost

In [ ]:
from catboost import CatBoostClassifier

Гиперпараметры я не буду подбирать, так как это долго.

In [ ]:
cb_clf = CatBoostClassifier(verbose=False, n_estimators=500, max_depth=6, min_child_samples=5, learning_rate=0.7)

start = time.time() #Начало времени обучения

cb_clf.fit(x_train_sample, y_train_sample)

finish = time.time() # Окончение обучения

startp = time.time() #Начало времени предсказания

y_pred = cb_clf.predict(x_test_sample)

finishp = time.time() # Окончение предсказания

print(f'Test score of Catboost = {accuracy_score(y_test_sample, y_pred)},fit time {finish-start:.2} second, predict time {finishp-startp:.2} second')

Test score of Catboost = 0.8456,fit time 9.6e+02 second, predict time 0.62 second


Подведем итогы:

1) Лучше всех оказалась логистическая регрессия с RFF, так как он имеет более высокое accuracy (около 0.85), по скоростьи он на 3 метсе, обучался около 50 секунд.

2) На втором месте Catboost с accuracy 0.84, но обучение самое долгое, около 15 минут, несмотря на то, что обучался на 12% выборки.

3) Потом по точности лучшее всех ядровой SVM, около 0.84. По скорости обучения занимает 1 место, аж 6 секунд, но на части данных.

4) Хуже всех по accuracy оказался обычный линейный svm, около 0.77, но по скорости быстрее всех - 7 секунд (тоже на части данных).

Скорости предсказания: \\
1) На первом месте ГБ аж 0.6 секунд \\
2) Логистическая регрессия с RFF 0.7 секнуд \\
3) Линейный SVM 2 секунда \\
4) Ядровой SVM 3 секунды

SVM-ы без урезанной выборки обучались дольше 3-5 миниут, и тем самым уже проигрывают логрегу с RFF. Поэтому оставил урезанную, чтобы при частом запуске не ждать долго.


__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA?
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

Начнем с первого пункта

In [ ]:
pipeline_without_pca = RFFPipeline(n_features=1000, new_dim=50, use_PCA=False, feature_creator_class=RandomFeatureCreator)

pipeline_without_pca.fit(x_train, y_train)

y_pred = pipeline_without_pca.predict(x_test)
print(f'Test score on RFF without PCA = {accuracy_score(y_test, y_pred)}')

Test score on RFF without PCA = 0.1089


Как мы видим без PCA accuracy упал 8 раз. То есть PCA помогает нам, он необхадим.

Второй пункт

In [ ]:
pipeline_new = RFFPipeline(n_features=200, new_dim=50, use_PCA=True, feature_creator_class=RandomFeatureCreator) #Уменьшим c 1000 до 200 n_feature

pipeline_new.fit(x_train, y_train)

y_pred = pipeline_new.predict(x_test)
print(f'Test score on RFF with 200 n_feature = {accuracy_score(y_test, y_pred)}')

Test score on RFF with 200 n_feature = 0.8454


C уменьшем n_feature accuracy падает, теперь попробуем увеличить

In [ ]:
pipeline_new = RFFPipeline(n_features=2000, new_dim=50, use_PCA=True, feature_creator_class=RandomFeatureCreator) #Увелечививаем c 1000 до 2000 n_feature

pipeline_new.fit(x_train, y_train)

y_pred = pipeline_new.predict(x_test)
print(f'Test score on RFF with 2000 n_feature = {accuracy_score(y_test, y_pred)}')

Test score on RFF with 2000 n_feature = 0.8609


In [ ]:
pipeline_new = RFFPipeline(n_features=4000, new_dim=50, use_PCA=True, feature_creator_class=RandomFeatureCreator) #Увелечививаем c 2000 до 4000 n_feature

pipeline_new.fit(x_train, y_train)

y_pred = pipeline_new.predict(x_test)
print(f'Test score on RFF with 4000 n_feature = {accuracy_score(y_test, y_pred)}')

Test score on RFF with 4000 n_feature = 0.861


Как видим, с увеличением растет accuracy, но выходит на плато, поэтому n_feature должно быть большим, но не сильно

Третий пункт

Попробуем линейным SVM

In [ ]:
pipeline_svm = RFFPipeline(n_features=1000, new_dim=50, use_PCA=True, feature_creator_class=RandomFeatureCreator, classifier_class=SVC, classifier_params={'kernel': 'linear'})

start = time.time() #Начало времени обучения

pipeline_svm.fit(x_train, y_train)

finish = time.time() #Конец времени обучения

startp = time.time() #Начало времени предсказания

y_pred = pipeline_svm.predict(x_test)

finishp = time.time() # Окончение предсказания

print(f'Test score on RFF with linear SVM = {accuracy_score(y_test, y_pred)}, fit time {finish-start:.2} second, predict time {finishp-startp:.2} second')

Test score on RFF with linear SVM = 0.8796, fit time 5.3e+02 second, predict time 1.1e+02 second


По части качества SVM c RFF гораздо лучше, чем логрег с RFF, но по части скорости обучения и предсказания в 10 раз уступает логрегу.

__Задание 5. (Максимум 1 балл)__

Существует большое количество работ, где идея RFF развивается, предлагаются её обобщения (которые, по сути, выливаются в другие преобразования признаков, не обязательно уже тригонометрические). Возьмите любую из таких работ, кратко опишите идею, имплементируйте её и сравните качество с ORF и RFF, которые вы запрограммировали выше.

Ссылки на статьи, где обсуждаются вариации RFF для разных ядер, можно найти в окрестности таблицы 1 в работе https://arxiv.org/pdf/1407.5599

Я решил рассматреть полиномиальное ядро

___ссылка на работу:___ https://arxiv.org/pdf/2212.07658

___описание идеи:___ Основная идея заключается в том, что если данные не разделяются линейно в исходном пространстве, то с помощью полиномиального ядра их можно перенести в пространство более высокой размерности, где разделение становится возможным.

Ядро выглядит следующим образом

$$
K(x,z) = (x^{T}z + c)^{d}
$$

где *с* и *d* являются гиперпараметрами.

Он уже релизован в sklearn, поэтому сразу можно использовать его.

In [ ]:
svc_poly = SVC(kernel='poly', degree = 3, coef0=0.5)

start = time.time() #Начало времени обучения

svc_poly.fit(x_train, y_train)

finish = time.time() #Конец времени обучения

startp = time.time() #Начало времени предсказания

y_pred = svc_poly.predict(x_test)

finishp = time.time() # Окончение предсказания

print(f'Test score on SVM with poly kernel = {accuracy_score(y_test, y_pred)}, fit time {finish-start:.2} second, predict time {finishp-startp:.2} second')

Test score on SVM with poly kernel = 0.8875, fit time 3.2e+02 second, predict time 1.1e+02 second


По результату SVM c полиномиальным ядром лучше чем логрег и linear SVM с RFF. По скорости он явно медленее логрега с RFF, но быстрее SVM-a (который тоже с RFF). Обучился за 5 минут, а предсказивал за 2 минуты.

__Задание 6. (Максимум 2.5 балла)__

Реализуйте класс ядровой Ridge регрессии (Лекция 13, $\S 1.2$), для оптимизации используте градиентный спуск **[1 балл максимум]**, также добавьте возможность использовать аналитическую формулу **[1 балл максимум]**. Для градиентного спуска выпишите градиент ниже **[0.5 баллов максимум]**.
Подумайте о том, как в формулах правильно учесть свободный коэффициент.

Затем адаптируйте вашу реализацию RFF под задачу регрессии. Сравните вашу ядровую регрессию и RFF на синтетических данных.

Функция потерь:
$$
Q(w) = \frac{1}{2} ||\Phi \Phi^T w - y||^2 + \frac{\lambda}{2} w^T \Phi \Phi^T w \rightarrow \min_w,
$$
где $\Phi \Phi^T = K$, $K = (k(x_i, x_j))_{i, j = 1}^{\ell}$.

Предсказание:
$
y(x) = k(x)^T w,
$
где $k(x)$ — вектор функций ядра от пар объектов $(x, x_i)_{i=1}^{\ell}$.

___Выведите градиент:___
$$
\nabla_{w}Q(w) = \Phi\Phi^{T}(\Phi\Phi^{T}w-y) + \lambda  \Phi\Phi^{T}w
$$

Вы можете изменять представленный шаблон в файле `homework_practice_08_kernel_regression.py` по своему усмотрению.

Я сразу же написал $$ \Phi\Phi^{T} $$ так как матрица симметричная и

$$
\Phi\Phi^{T} = (\Phi\Phi^{T})^{T}
$$

In [ ]:
from homework_practice_08_kernel_regression_SargsyanNarek import KernelRidgeRegression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Начнем с синтетических данных

In [ ]:
lin_reg = KernelRidgeRegression(regularization=10, lr = 0.004, max_iter=500, batch_size=64, kernel_scale = 18)

In [ ]:
from sklearn.datasets import make_regression
np.random.seed(152)
X, y = make_regression(n_samples=1500, n_features=15)

# Шаг 2: Разделение данных на train и test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=153)

In [ ]:
lin_reg.fit_closed_form(X_train, y_train)

In [ ]:
pred = lin_reg.predict(X_test)

In [ ]:
print(f'MSE on train {mean_squared_error(y_train, lin_reg.predict(X_train))}')
print(f'R2 on train {r2_score(y_train, lin_reg.predict(X_train))}')
print(f'MSE on test {mean_squared_error(y_test, pred)}')
print(f'R2 on test {r2_score(y_test, pred)}')

MSE on train 3.55412493114913
R2 on train 0.9998865378274284
MSE on test 3.5571434584369257
R2 on test 0.9998940928767662


Если обучать дефолтными переметрами, то там будет жуткое переобучение, поэтому важно подобрать гиперпараметры. Также важно отметить, что обучается около 40 секунд.


Попробуем теперь линрег с градиентным спуском

In [ ]:
import numpy as np

In [ ]:
lin_reg = KernelRidgeRegression(regularization=4, lr = 0.004, max_iter=20000, batch_size=64, kernel_scale = 5)

In [ ]:
lin_reg.fit(X_train, y_train)
pred = lin_reg.predict(X_test)

In [ ]:
print(f'MSE on train {mean_squared_error(y_train, lin_reg.predict(X_train))}')
print(f'R2 on train {r2_score(y_train, lin_reg.predict(X_train))}')
print(f'MSE on test {mean_squared_error(y_test, pred)}')
print(f'R2 on test {r2_score(y_test, pred)}')

MSE on train 2266.8708377156668
R2 on train 0.9276322315143873
MSE on test 2950.4674939092383
R2 on test 0.9121554898963571


Он как то очень медленно идет к миниммуму, но в целом работает. Какой нибудь шкедулер или другой метод градиентного спуска дали бы лучше результат.

Теперь сгенерируем датасет и спользуем логрег с RFF

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

np.random.seed(152)

X_linear, y_linear = make_classification(n_samples=1500, n_features=17, n_classes=2,
                                         n_informative=2, n_redundant=0, random_state=42)

X_train_lin, X_test_lin, y_train_lin, y_test_lin = train_test_split(X_linear, y_linear, test_size=0.2, random_state=42)

In [ ]:
pipe_syn = RFFPipeline(n_features=10).fit(X_train_lin, y_train_lin)

y_pred = pipe_syn.predict(X_test_lin)

print(f"Train score = {accuracy_score(y_train_lin, pipe_syn.predict(X_train_lin)):.2}")
print(f"Test score = {accuracy_score(y_test_lin, y_pred):.2}")

Train score = 0.89
Test score = 0.87


С задачей классификации никаких проблем нет, все отлично предсказывает.